# Tapering off qubits using spatial symmetries

The following code demonstrates the procedure to taper off qubits using point group symmetries present in the molecules. This notebook is part of the repository mole_geo_symmetries hosted on github. The repository is compatible only with qiskit-0.11.0 
There are three main files that the following code uses:

1. r_mat_for_mols_old.py: This file is for running the pyscf for getting the integrals for a given molecule and the R matrices for the various symmetries present in the molecule.

2. taper_qubits_rm_funcs_old.py: This is the file with all the subroutines for tapering off the qubits using the point group symmetries. This file also contains a script that runs through the whole procedure for a given molecule. Part of this script is used in this jupyter notebook to demonstrate the procedure.

3. int_func.py: This is a helper file which is a modified version of the qiskit driver for pyscf. This was developed to get the atomic integrals.

The code below is structure as follows:
- import statements
- setup for a given molecule and getting an object that contains R matrices, fermionicoperator. The R matrices and fermionic operators are used to get the V matrix.
- The fermionicoperator is transformed using Jordan-Wigner mapping to qubit operator(qub_op). A copy of fermionicoperator is transformed using the v matrix. Another qubit operator (v_qubit_op) is obtained using Jordan-Wigner mapping on the transformed Hamiltonian.
- Exact diagonalization is performed on the qub_op to get the reference eigenvalues.
- The symmetries are constructed from the transformed R-matrices.
- As a sanity check, it is verified that these symmetries commute with the Hamiltonian.
- The cliffords are obtained and the qubits are tapered off. This is an iterative process as we do not know the correct eigen sector.
- The correct tapered off qubit operator is printed along with the lowest eigenvalues.
- Finally, the fermionic operator and qubit operator are obtained in the molecular orbital basis and the lowest eigenvalue are obtained as another sanity check for the values obtained from the AO basis.


In [1]:
# importing stuff
import logging
import copy
import itertools
from qiskit.quantum_info import Pauli
from pyscf import gto, scf, ao2mo
from pyscf.lib import param
from scipy import linalg as scila
from pyscf.lib import logger as pylogger
from qiskit.chemistry import QMolecule
import numpy as np
from qiskit.aqua import Operator
from qiskit.aqua.algorithms import ExactEigensolver
import scipy
from pyscf.scf.hf import get_ovlp
from symmetries import find_symmetry_ops
from qiskit.chemistry import FermionicOperator
from int_func import qmol_func
logger = logging.getLogger(__name__)
import int_func
from int_func import qmol_func
from taper_qubits_rm_funcs_old import *
from r_mat_for_mols_old import mol_r_matrices, check_commute
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)


/home/k/quantum_code/test_env/lib/python3.6/site-packages/qutip/__init__.py:161: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


# AO basis setup

In [2]:
# Change the following flag to true to print the R matrices.
# The order of the 
FLAG_PRINT_R_MATRICES = False
AO = True
# R matrices have been coded for, 'H2', 'H2O_l', 'H2O', 'NH3', 'BeH2', 'C2H2', 'C2H4', 'LiH'
molecule = 'BeH2'
# The following flag is for checking if all the r matrices commute with the Hamiltonian.
# All the r matrices in the file r_mat_for_mols_old .py have been checked. If new r_matrices are added 
# then the flag could be turned to True.
check_r_mat_commut=False
# This calculates the reference eigenspectrum. Make the following
# flag true, only when the system is small.
check_ref_energy = True
# r_mat_func 
x = r_mat_funcs(molecule, check_r_mat_commut,AO)
# printing the R-matrices for BeH2
counter = 1
if FLAG_PRINT_R_MATRICES:
    for i in x.r_matrices:
        print('R'+str(counter)+' = ')
        print(i)
        counter+=1
# The R-matrices are simultaneously diagonalized
[r_mat_evals,v_matrix] = x.sim_diag(x.r_matrices)


BeH2
converged SCF energy = -15.5613526278409


/home/k/quantum_code/test_env/lib/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:


checking the v matrix...
v matrix is  unitary.


In [3]:
# Getting the qubit opertor form of Hamiltonian
qub_op = x.fer_op.mapping('jordan_wigner')
# Total number of terms in the qubit operator Hamiltonian
qub_op.chop()
print('Number of terms in the Hamiltonian in AO basis')
print(len(qub_op._paulis))

# Getting and transforming the Ham with V matrix.
v_qubit_op = x.sym_transf_ham_qub_op(v_matrix)

#Printing the number of terms in the Hamiltonian
print('Final number of terms in the Hamiltonian in AO basis after the transformation')
print(len(v_qubit_op._paulis))

Number of terms in the Hamiltonian in AO basis
1150
Final number of terms in the Hamiltonian in AO basis after the transformation
666


In [4]:
# Sanity check. Calculate the spectrum of Hamiltonian
if check_ref_energy ==True:
    ee = ExactEigensolver(qub_op, k=3)
    ee_result = ee.run()
#   Get the first three eigenvalues of the Hamiltonian spectrum
    ref_min_eigvals = ee_result['eigvals'][0:3]
    # This is the reference value from Hamiltonian in AO basis
    print('Eigen value of the full Ham in AO basis')
    print(ref_min_eigvals)

# The symmetries are not independent, so, the following code gets the independent set of symmetries.
r_mat_evals = x.ind_symm_r_ev_mat(r_mat_evals)
sym_list = x.get_symm_list(r_mat_evals)



Eigen value of the full Ham in AO basis
[-19.079-0.j -18.877+0.j -18.877+0.j]


In [6]:
# In order to check if the symmetries commute with Ham, uncomment the following piece of code:
print("check the commutativity of the found symmetry paulis between H'.")
for symm in sym_list:
    symm_op = Operator(paulis=[[1.0, symm]])
    is_commutes = check_commute(symm_op, v_qubit_op)
    print(symm_op.print_operators())
    # symm_op.to_matrix()
    # print('Trace of the operators')
    # print(np.trace(symm_op._matrix.todense()))
    sym_la = symm.to_label()[::-1]
    ind = [i for i, a in enumerate(sym_la) if a == 'Z']
    print(ind)
    print("symmetry {} commutes with the Hamiltonian.".format("" if is_commutes else "NOT"))

# exit()
# Get the unitary operators (cliffords) corresponding the single qubit string.



check the commutativity of the found symmetry paulis between H'.
ZZIIIIIIIIIIII	1.0

[12, 13]
symmetry  commutes with the Hamiltonian.
IIZZIIIIIIIIII	1.0

[10, 11]
symmetry  commutes with the Hamiltonian.
IIIIZZZZIIIIII	1.0

[6, 7, 8, 9]
symmetry  commutes with the Hamiltonian.
ZIZIZZIIZZZIII	1.0

[3, 4, 5, 8, 9, 11, 13]
symmetry  commutes with the Hamiltonian.
ZIZIZZIIIIIZZZ	1.0

[0, 1, 2, 8, 9, 11, 13]
symmetry  commutes with the Hamiltonian.


In [8]:
[cliffords, single_qubit_list] = x.get_cliffords(r_mat_evals,sym_list)
print('Following are the qubits which are tappered off.')
print(single_qubit_list)

print("Trying to tapering")
correct_sector = None
for taper_coeff in itertools.product([1, -1], repeat=len(single_qubit_list)):
    tapered_qubit_op = Operator.qubit_tapering(v_qubit_op, cliffords, single_qubit_list, list(taper_coeff))
    ee = ExactEigensolver(tapered_qubit_op, k=1)
    ee_result = ee.run()
    temp_min_eigvals = ee_result['eigvals'][0]
    if np.isclose(temp_min_eigvals, ref_min_eigvals[0], rtol=1e-8):
        correct_sector = list(taper_coeff)
    print("at sector {}: eig value: {}; reference: {}".format(list(taper_coeff), temp_min_eigvals, ref_min_eigvals.real))

# correct_sector=[1.,1.,1.,-1.,-1.]

# Get the tappered qubit operator
tapered_qubit_op = x.get_tapered_qubit_op(v_qubit_op,cliffords,single_qubit_list,correct_sector)
ee = ExactEigensolver(tapered_qubit_op.copy(), k=6)
ee_result = ee.run()
print('Getting the eigen values of the tappered off qubit operator')
print(ee_result['eigvals'][0:6])


Following are the qubits which are tappered off.
[12, 10, 6, 3, 0]
Trying to tapering
at sector [1, 1, 1, 1, 1]: eig value: (-18.56322747526113-1.769417815513804e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, 1, 1, -1]: eig value: (-18.63558372129266+2.5096035288891392e-17j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, 1, -1, 1]: eig value: (-18.63558372129268+2.3978979155952983e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, 1, -1, -1]: eig value: (-19.078889372592677+1.6485125388163513e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, -1, 1, 1]: eig value: (-18.65675954846912+3.327082406515677e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, -1, 1, -1]: eig value: (-18.645545229515033+1.6182317017319728e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, -1, -1, 1]: eig value: (-18.645545229514955-2.706763704409162e-16j); reference: [-19.079 -18.877 -18.877]
at sector [1, 1, -1, -1, -1]: eig value: (-18.656759548469

# Molecular orbital basis


In [9]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.chemistry import QMolecule
import warnings
from qiskit.chemistry.fermionic_operator import FermionicOperator
warnings.filterwarnings("ignore",category=DeprecationWarning)

FLAG_PRINT_R_MATRICES = False
AO = True
# R matrices have been coded for, 'H2', 'H2O_l', 'H2O', 'NH3', 'BeH2', 'C2H2', 'C2H4', 'LiH'
molecule = 'BeH2'
# The following flag is for checking if all the r matrices commute with the Hamiltonian.
# All the r matrices in the file r_mat_for_mols_old .py have been checked. If new r_matrices are added 
# then the flag could be turned to True.
check_r_mat_commut=False
# This calculates the reference eigenspectrum. Make the following
# flag true, only when the system is small.
check_ref_energy = True
# r_mat_func 
x = r_mat_funcs(molecule, check_r_mat_commut,AO)
qub_op = x.fer_op.mapping('jordan_wigner')
# Total number of terms in the qubit operator Hamiltonian
qub_op.chop()

ee = ExactEigensolver(qub_op, k=6)
ee_result = ee.run()
ref_min_eigvals = ee_result['eigvals'][0:6]
print(ref_min_eigvals)

BeH2
converged SCF energy = -15.5613526278408


/home/k/quantum_code/test_env/lib/python3.6/site-packages/pyscf/scf/chkfile.py:31: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(chkfile) as fh5:


[-19.079-0.j -18.877+0.j -18.877+0.j -18.877+0.j -18.877-0.j -18.813+0.j]
